# 1. Introduction
**FastText** is an open-source, text classification library created by **Facebook's AI Research (FAIR)** team. It provides an efficient and fast way to classify text data.

A *FastText News Classifier* is a machine learning model that leverages **FastText** to classify news articles into different categories (such as sports, politics, entertainment, etc.).

**Thanks to:**
* [E-commerce Text Classification Using FastText](https://www.kaggle.com/code/sunilthite/e-commerce-text-classification-using-fasttext/input)

# 2. Import libraries

In [1]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296188 sha256=8e7e18ad196ae97dc37f550167032f1c40549b5682322ee70f30b5cf6bf68616
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [3]:
# FastText library for text classification and training the FastText model.
import fasttext

# pandas library for data manipulation.
import pandas as pd